In [1]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM, Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l2

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Code

In [2]:
data_path = pd.read_csv("data_path.csv")

In [50]:
# Load the ravdess data
ravdess = pd.read_csv("Data/Ravdess.csv")
ravdess = ravdess[(ravdess.Emotions != 'neutral')]
data_path = ravdess

In [58]:
tess = pd.read_csv("Data/Tess.csv")
data_path = tess

In [16]:
crema_d = pd.read_csv("Data/Crema.csv")
data_path = crema_d

In [43]:
savee = pd.read_csv("Data/Savee.csv")
savee = savee[(savee.Emotions != 'neutral')]
data_path = savee

In [3]:
# I want to remove suprise and calm emotions from the dataset
data_path = data_path[(data_path.Emotions != 'calm') & (data_path.Emotions != 'surprise')]

In [ ]:
plt.title('Count of Emotions', size=16)
sns.countplot(data_path.Emotions)
plt.ylabel('Count', size=12)
plt.xlabel('Emotions', size=12)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.show()

In [19]:
# Padding function
def pad_features(features, max_len=100):
    return pad_sequences(features, maxlen=max_len, padding='post', dtype='float32')

In [20]:
X, Y = [], []
total_duration = 0
file_count = 0

for path, emotion in zip(data_path.Path, data_path.Emotions):
    data, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13).T
    X.append(mfcc)       
    Y.append(emotion)
    
X = pad_features(X, max_len=100)

Ravdess = 3.71
TESS = 2.05
CremaD = 2.54
Savee = 3.91

In [21]:
# Convert X and Y to numpy arrays (if not already)
X = np.array(X)
Y = np.array(Y)

In [7]:
import numpy as np

# Convert X and Y to numpy arrays (if not already)
X = np.array(X)
Y = np.array(Y)

# Save X and Y as .npy files
np.save('X_features.npy', X)
np.save('Y_labels.npy', Y)


In [8]:
np.save('X_mfcc.npy', X)
np.save('Y_mfcc_labels.npy', Y)

In [ ]:
len(X), len(Y), data_path.Path.shape

In [ ]:
X

In [ ]:
X.shape

# Implement from here

In [5]:
# Load saved X and Y
X = np.load('X_mfcc.npy', allow_pickle=True)
Y = np.load('Y_mfcc_labels.npy', allow_pickle=True)

In [ ]:
Y.shape

In [ ]:
# I want a list that has the unique emotions
unique_emotions = data_path.Emotions.unique()
# unique_emotions
# convert to list
unique_emotions = list(unique_emotions)
unique_emotions
# count the number of unique emotions
n_classes = len(unique_emotions)
n_classes, unique_emotions
emotions = np.array(unique_emotions)
emotions

In [7]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()

# Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
encoded_labels = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.3, random_state=42, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
x_train.shape[2]

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense,GlobalMaxPooling1D, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2

def build_model(x_train, y_train, x_test, y_test, num_classes):
    
    input_shape = (100, x_train.shape[2])
    audio_model = Sequential([
        # Convolutional layer
        Conv1D(filters=64, kernel_size=3, activation='relu', 
               kernel_regularizer=l2(0.01), padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        Conv1D(filters=128, kernel_size=3, activation='relu', 
               kernel_regularizer=l2(0.01), padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        # BiLSTM layer
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        
        Bidirectional(LSTM(64)),
        Dropout(0.2),

        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    audio_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = audio_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, min_delta=0.001, mode='min', restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath='best_mfcc_model.keras', monitor='val_loss', save_best_only=True)
    ])

    # Evaluate the model on the test set
    loss, accuracy = audio_model.evaluate(x_test, y_test)
    print(f"Audio Model Test Accuracy: {accuracy * 100:.2f}%")

    return audio_model, history

# Train the audio model with pre-extracted features
audio_model, history = build_model(x_train, y_train, x_test, y_test, num_classes=n_classes)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

def analyze_emotion_classification(model, x_test, y_test, class_names):
    # Get predictions
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # 1. Class-wise Performance Metrics
    print("Detailed Classification Report:")
    print(classification_report(true_classes, predicted_classes, target_names=class_names))
    
    # 2. Confusion Matrix with custom formatting
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(true_classes, predicted_classes)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Normalized Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 3. Per-class Performance Analysis
    class_metrics = {
        'Precision': [],
        'Recall': [],
        'F1-Score': []
    }
    
    for i in range(len(class_names)):
        true_class = (true_classes == i)
        pred_class = (predicted_classes == i)
        
        tp = np.sum(true_class & pred_class)
        fp = np.sum(~true_class & pred_class)
        fn = np.sum(true_class & ~pred_class)
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        class_metrics['Precision'].append(precision)
        class_metrics['Recall'].append(recall)
        class_metrics['F1-Score'].append(f1)
    
    # Plot per-class metrics
    metrics_df = pd.DataFrame(class_metrics, index=class_names)
    
    plt.figure(figsize=(12, 6))
    metrics_df.plot(kind='bar', width=0.8)
    plt.title('Per-class Performance Metrics')
    plt.xlabel('Emotion')
    plt.ylabel('Score')
    plt.legend(loc='lower right')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 4. Emotion Correlation Analysis
    emotion_probs = pd.DataFrame(predictions, columns=class_names)
    correlation_matrix = emotion_probs.corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdBu_r',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Emotion Prediction Correlation Matrix')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return metrics_df, correlation_matrix

# Usage
# Assuming you have your class names in a list
class_names = unique_emotions  # replace with your actual emotion classes
metrics_df, correlation_matrix = analyze_emotion_classification(audio_model, x_test, y_test, class_names)

# Temporary

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency
import networkx as nx
from scipy.cluster.hierarchy import dendrogram, linkage

class EmotionRelationshipAnalysis:
    def __init__(self, model, x_test, y_test, class_names):
        self.model = model
        self.x_test = x_test
        self.y_test = y_test
        self.class_names = class_names
        self.predictions = model.predict(x_test)
        self.true_labels = np.argmax(y_test, axis=1)
        self.pred_labels = np.argmax(self.predictions, axis=1)
        
    def analyze_confusion_patterns(self):
        """Analyze and visualize confusion patterns between emotions"""
        # Create confusion matrix
        cm = confusion_matrix(self.true_labels, self.pred_labels)
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                    xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title('Normalized Confusion Matrix: Emotion Misclassification Patterns')
        plt.ylabel('True Emotion')
        plt.xlabel('Predicted Emotion')
        plt.tight_layout()
        plt.show()
        
        # Analyze most common confusions
        confusion_pairs = []
        for i in range(len(self.class_names)):
            for j in range(len(self.class_names)):
                if i != j:
                    confusion_pairs.append({
                        'True': self.class_names[i],
                        'Predicted': self.class_names[j],
                        'Rate': cm_normalized[i, j]
                    })
        
        confusion_df = pd.DataFrame(confusion_pairs)
        print("\nTop Confusion Pairs:")
        print(confusion_df.sort_values('Rate', ascending=False).head())
        
        return cm_normalized, confusion_df
    
    def emotion_correlation_network(self):
        """Create and visualize emotion correlation network"""
        # Calculate correlation between prediction probabilities
        emotion_probs = pd.DataFrame(self.predictions, columns=self.class_names)
        correlations = emotion_probs.corr()
        
        # Create network graph
        plt.figure(figsize=(12, 12))
        G = nx.Graph()
        
        # Add nodes and edges
        for i in range(len(self.class_names)):
            G.add_node(self.class_names[i])
            for j in range(i+1, len(self.class_names)):
                if abs(correlations.iloc[i, j]) > 0.1:  # Threshold for visualization
                    G.add_edge(self.class_names[i], self.class_names[j],
                             weight=abs(correlations.iloc[i, j]))
        
        # Draw network
        pos = nx.spring_layout(G)
        edges = G.edges()
        weights = [G[u][v]['weight'] * 2 for u, v in edges]
        
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                             node_size=1000, alpha=0.7)
        nx.draw_networkx_edges(G, pos, width=weights, alpha=0.5)
        nx.draw_networkx_labels(G, pos)
        
        plt.title("Emotion Relationship Network")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        return correlations
    
    def emotion_clustering(self):
        """Perform hierarchical clustering of emotions"""
        # Get feature representations for each emotion
        emotion_features = []
        for i in range(len(self.class_names)):
            mask = (self.true_labels == i)
            emotion_features.append(np.mean(self.x_test[mask], axis=0))
        
        # Perform hierarchical clustering
        linkage_matrix = linkage(emotion_features, 'ward')
        
        plt.figure(figsize=(10, 7))
        dendrogram(linkage_matrix, labels=self.class_names)
        plt.title('Hierarchical Clustering of Emotions')
        plt.xlabel('Emotions')
        plt.ylabel('Distance')
        plt.tight_layout()
        plt.show()
        
        return linkage_matrix
    
    def analyze_feature_importance_per_emotion(self):
        """Analyze which features are most important for each emotion"""
        feature_importance = {}
        
        for i, emotion in enumerate(self.class_names):
            # Get samples for this emotion
            mask = (self.true_labels == i)
            emotion_samples = self.x_test[mask]
            
            # Calculate mean feature values
            mean_features = np.mean(emotion_samples, axis=0)
            std_features = np.std(emotion_samples, axis=0)
            
            # Store importance scores
            feature_importance[emotion] = {
                'mean': mean_features,
                'std': std_features
            }
        
        # Visualize feature importance
        plt.figure(figsize=(15, 8))
        for emotion in self.class_names:
            plt.plot(feature_importance[emotion]['mean'], label=emotion)
        
        plt.title('Feature Patterns Across Emotions')
        plt.xlabel('Feature Index')
        plt.ylabel('Mean Feature Value')
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        return feature_importance
    
    def transition_analysis(self):
        """Analyze temporal transitions between emotions"""
        # Create transition matrix
        n_emotions = len(self.class_names)
        transitions = np.zeros((n_emotions, n_emotions))
        
        for i in range(len(self.pred_labels)-1):
            transitions[self.pred_labels[i], self.pred_labels[i+1]] += 1
        
        # Normalize
        row_sums = transitions.sum(axis=1)
        transitions_normalized = transitions / row_sums[:, np.newaxis]
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(transitions_normalized, annot=True, fmt='.2f', cmap='Blues',
                    xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title('Emotion Transition Probabilities')
        plt.ylabel('Current Emotion')
        plt.xlabel('Next Emotion')
        plt.tight_layout()
        plt.show()
        
        return transitions_normalized

# Example usage:

# Initialize analyzer
analyzer = EmotionRelationshipAnalysis(audio_model, x_test, y_test, class_names)

# 1. Analyze confusion patterns
cm_normalized, confusion_df = analyzer.analyze_confusion_patterns()

# 2. Create emotion correlation network
correlations = analyzer.emotion_correlation_network()

# 3. Perform emotion clustering
# linkage_matrix = analyzer.emotion_clustering()

# 4. Analyze feature importance per emotion
feature_importance = analyzer.analyze_feature_importance_per_emotion()

# 5. Analyze emotion transitions
transitions = analyzer.transition_analysis()


# Outputs

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Assuming `Y` is the original list of labels (before one-hot encoding)
# Fit the LabelEncoder on the original labels
label_encoder = LabelEncoder()
label_encoder.fit(np.argmax(y_train, axis=1))  # Fit LabelEncoder on the integer-encoded classes

# Since `y_test` is one-hot encoded, convert it back to the original class labels
y_true = np.argmax(y_test, axis=1)

# Predict the labels for the test set
y_pred = audio_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions from one-hot encoding to class labels

# Function to plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# Get the unique class labels using the fitted label encoder
class_labels = label_encoder.classes_

# Plot the confusion matrix
plot_confusion_matrix(y_true, y_pred_classes, class_labels=class_labels)


In [ ]:
import matplotlib.pyplot as plt

# Function to plot the training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.show()

# Plot the training history
plot_training_history(history)


In [18]:
# import umap
# import matplotlib.pyplot as plt

# # Initialize UMAP for 2D
# umap_2d = umap.UMAP(n_components=2, random_state=42)

# flat_features = X.reshape(X.shape[0], -1)

# # Fit and transform the flattened features
# umap_features_2d = umap_2d.fit_transform(flat_features)

# # Create the scatter plot
# plt.figure(figsize=(10, 8))
# sns.scatterplot(x=umap_features_2d[:, 0], y=umap_features_2d[:, 1], hue=encoded_labels, palette='viridis', s=60)

# # Add labels and title
# plt.title('UMAP Visualization of Features Based on Emotions')
# plt.xlabel('UMAP Feature 1')
# plt.ylabel('UMAP Feature 2')
# plt.legend(title='Emotion', labels=unique_emotions)
# plt.show()


# Visualization

In [ ]:
data_path

In [ ]:
# Group by the 'Emotions' column and pick the first sample for each emotion
samples = data_path.groupby('Emotions').first().reset_index()

# Display the resulting dataframe
print(samples)

In [ ]:
samples['Emotions'][0]

# SECOND ATTEMPT

In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display

audio_file = samples['Path'][0]
y, sr = librosa.load(audio_file, sr=16000)
plt.figure(figsize=(12, 4))
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform of the Audio Sample')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
plt.figure(figsize=(12, 6))
librosa.display.specshow(mfccs, sr=sr, x_axis='time', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('MFCC Coefficients')
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_file = 'your_audio_sample.wav'
y, sr = librosa.load(audio_file, sr=None)

# Remove silence at the start and end
y, _ = librosa.effects.trim(y)

# Create time axis for the waveform and spectrogram
duration = librosa.get_duration(y=y, sr=sr)
time = np.linspace(0., duration, len(y))

# Create the figure and axis for both plots
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Plot the waveform
ax[0].plot(time, y)
ax[0].set(title='Waveform', xlabel='Time (s)', ylabel='Amplitude')

# Plot the spectrogram
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', ax=ax[1])
ax[1].set(title='Spectrogram', xlabel='Time (s)', ylabel='Frequency (Hz)')
ax[1].colorbar(format='%+2.0f dB')

# Adjust the layout
plt.tight_layout()
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_file = samples['Path'][0]  # Replace with your audio file path
y, sr = librosa.load(audio_file, sr=None)

# Remove silences at the start and end
y_trimmed, _ = librosa.effects.trim(y)

# Create the figure with 3 subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Plot the waveform
axs[0].set_title('Waveform (Trimmed)')
librosa.display.waveshow(y_trimmed, sr=sr, ax=axs[0])
axs[0].set_ylabel('Amplitude')

# Plot the Mel spectrogram
axs[1].set_title('Mel Spectrogram')
S = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128, fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)
img1 = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=axs[1], cmap='magma')
axs[1].set_ylabel('Frequency (Hz)')
fig.colorbar(img1, ax=axs[1], format="%+2.0f dB")

# Plot the MFCC spectrogram
axs[2].set_title('MFCC Spectrogram')
MFCCs = librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=13)
img2 = librosa.display.specshow(MFCCs, x_axis='time', sr=sr, ax=axs[2], cmap='viridis')
axs[2].set_ylabel('MFCC Coefficients')
fig.colorbar(img2, ax=axs[2], format="%+2.0f dB")

# Add a shared X-axis label
plt.xlabel('Time (s)')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_file = samples['Path'][0]  # Replace with your audio file path
y, sr = librosa.load(audio_file, sr=None)

# Remove silences at the start and end
y_trimmed, _ = librosa.effects.trim(y)

# Calculate time array for waveform
time = np.linspace(0, len(y_trimmed) / sr, len(y_trimmed))

# Create Mel spectrogram
S = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128, fmax=8000)

# Create MFCC spectrogram
MFCCs = librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=13)

# Create spectrogram using STFT
D = librosa.amplitude_to_db(np.abs(librosa.stft(y_trimmed)), ref=np.max)

# Create the figure with 4 subplots
fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)

# Plot the waveform
axs[0].set_title('Waveform')
axs[0].plot(time, y_trimmed, color='blue')
axs[0].set_ylabel('Amplitude')

# Plot the spectrogram
axs[1].set_title('Spectrogram (STFT)')
img1 = axs[1].imshow(
    D, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 0, sr // 2], 
    cmap='viridis'
)
axs[1].set_ylabel('Frequency (Hz)')
fig.colorbar(img1, ax=axs[1], format="%+2.0f dB")

# Plot the Mel spectrogram
axs[2].set_title('Mel Spectrogram')
mel_time = np.linspace(0, len(y_trimmed) / sr, S.shape[1])  # Align Mel spectrogram timeline
img2 = axs[2].imshow(
    librosa.power_to_db(S, ref=np.max), 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 0, 8000], 
    cmap='magma'
)
axs[2].set_ylabel('Frequency (Hz)')
fig.colorbar(img2, ax=axs[2], format="%+2.0f dB")

# Plot the MFCC spectrogram
axs[3].set_title('MFCC Spectrogram')
mfcc_time = np.linspace(0, len(y_trimmed) / sr, MFCCs.shape[1])  # Align MFCC spectrogram timeline
img3 = axs[3].imshow(
    MFCCs, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 1, 13], 
    cmap='plasma'
)
axs[3].set_ylabel('MFCC Coefficients')
fig.colorbar(img3, ax=axs[3], format="%+2.0f dB")

# Add a shared X-axis label
plt.xlabel('Time (s)')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_file = samples['Path'][0]  # Replace with your audio file path
y, sr = librosa.load(audio_file, sr=None)

# Remove silences at the start and end
y_trimmed, _ = librosa.effects.trim(y)

# Calculate time array for waveform
time = np.linspace(0, len(y_trimmed) / sr, len(y_trimmed))

# Create Mel spectrogram
S = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128, fmax=8000)

# Create MFCC spectrogram
MFCCs = librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=13)

# Create the figure with 3 subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Plot the waveform
axs[0].set_title('Waveform (Trimmed)')
axs[0].plot(time, y_trimmed, color='blue')
axs[0].set_ylabel('Amplitude')

# Plot the Mel spectrogram (raw values, no dB conversion)
axs[1].set_title('Mel Spectrogram (Raw)')
mel_time = np.linspace(0, len(y_trimmed) / sr, S.shape[1])  # Align Mel spectrogram timeline
img1 = axs[1].imshow(
    S, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 0, 8000], 
    cmap='magma'
)
axs[1].set_ylabel('Frequency (Hz)')
fig.colorbar(img1, ax=axs[1], format="%+2.0f")

# Plot the MFCC spectrogram (raw values)
axs[2].set_title('MFCC Spectrogram (Raw)')
mfcc_time = np.linspace(0, len(y_trimmed) / sr, MFCCs.shape[1])  # Align MFCC spectrogram timeline
img2 = axs[2].imshow(
    MFCCs, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 1, 13], 
    cmap='viridis'
)
axs[2].set_ylabel('MFCC Coefficients')
fig.colorbar(img2, ax=axs[2], format="%+2.0f")

# Add a shared X-axis label
plt.xlabel('Time (s)')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_path = samples['Path'][0]  # Replace with your audio file path
y, sr = librosa.load(audio_path, sr=None)

# Remove silences
y_trimmed, _ = librosa.effects.trim(y, top_db=20)  # Adjust top_db for sensitivity

# Create the Mel Spectrogram
S = librosa.feature.melspectrogram(y_trimmed, sr=sr, n_mels=128, fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)

# Plot the waveform and Mel spectrogram
fig, axs = plt.subplots(2, 1, figsize=(12, 6), gridspec_kw={'height_ratios': [1, 2]}, sharex=True)

# Plot the waveform
librosa.display.waveshow(y_trimmed, sr=sr, ax=axs[0], color='blue')
axs[0].set_title('Trimmed Waveform', fontsize=14)
axs[0].set_ylabel('Amplitude')
axs[0].grid()

# Plot the Mel Spectrogram
img = librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel', fmax=8000, ax=axs[1], cmap='viridis')
axs[1].set_title('Mel Spectrogram', fontsize=14)
axs[1].set_ylabel('Frequency (Hz)')
axs[1].grid()
fig.colorbar(img, ax=axs[1], format='%+2.0f dB')

# Set shared x-axis label
axs[1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()


In [ ]:
mfccs.shape

In [ ]:
import seaborn as sns
import pandas as pd

mfcc_df = pd.DataFrame(mfccs.T, columns=[f'MFCC {i+1}' for i in range(mfccs.shape[0])])
plt.figure(figsize=(12, 6))
sns.boxplot(data=mfcc_df)
plt.title('Boxplot of MFCC Coefficients')
plt.xlabel('MFCC Coefficient Index')
plt.ylabel('Value')
plt.show()


In [ ]:
# mean_mfcc.shape
X.shape

In [63]:
mfcc_features = X
labels = Y

In [ ]:
Y.shape

In [ ]:


# Compute mean across time steps for each sample
mean_mfcc = np.mean(mfcc_features, axis=1)  # Shape: (n_samples, n_coefficients)

# Convert to DataFrame for easier plotting
df = pd.DataFrame(mean_mfcc, columns=[f"MFCC {i+1}" for i in range(mean_mfcc.shape[1])])
df['Emotion'] = labels

# Melt for grouped bar plot
df_melted = df.melt(id_vars='Emotion', var_name='MFCC Coefficient', value_name='Mean Value')

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(data=df_melted, x='MFCC Coefficient', y='Mean Value', hue='Emotion', ci="sd")
plt.title('Mean MFCC Coefficients Across Emotions')
plt.xticks(rotation=45)
plt.legend(title='Emotion')
plt.show()

In [ ]:
# Select only the desired MFCC coefficients
selected_mfcc = ["MFCC 1", "MFCC 2", "MFCC 4", "MFCC 6"]
df_selected = df.melt(id_vars='Emotion', var_name='MFCC Coefficient', value_name='Mean Value')
df_selected = df_selected[df_selected['MFCC Coefficient'].isin(selected_mfcc)]

# Plot the selected coefficients
plt.figure(figsize=(12, 6))
sns.barplot(data=df_selected, x='MFCC Coefficient', y='Mean Value', hue='Emotion', ci="sd")
plt.title('Mean of Selected MFCC Coefficients Across Emotions')
plt.xticks(rotation=45)
plt.legend(title='Emotion')
plt.show()


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Flatten MFCC features (mean across time steps)
mfcc_flattened = np.mean(mfcc_features, axis=1)

# Standardize the features
scaler = StandardScaler()
mfcc_scaled = scaler.fit_transform(mfcc_flattened)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
mfcc_tsne = tsne.fit_transform(mfcc_scaled)

# Plot t-SNE results
plt.figure(figsize=(8, 7))
for emotion in emotions:
    idx = np.where(labels == emotion)
    plt.scatter(mfcc_tsne[idx, 0], mfcc_tsne[idx, 1], label=emotion, alpha=0.7)
plt.title('t-SNE Visualization of MFCC Features')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend(title='Emotion')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_mfcc_heatmap(mfcc_features, emotion_label, title="MFCC Heatmap"):
    plt.figure(figsize=(10, 4))
    plt.imshow(mfcc_features.T, aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(label="Magnitude")
    plt.title(f"{title}: {emotion_label}")
    plt.xlabel("Time Frames")
    plt.ylabel("MFCC Coefficients")
    plt.tight_layout()
    plt.show()

# Example usage
emotion_label = "Happy"
mfcc_features = np.random.rand(140, 13)  # Replace with real MFCC data
plot_mfcc_heatmap(mfcc_features, emotion_label)


In [13]:
# Load each audio file and extract MFCCs
mfcc_features = {}
for emotion, file_path in zip(samples["Emotions"], samples["Path"]):
    y, sr = librosa.load(file_path, sr=16000)  # Load the audio with 16kHz sampling rate
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract 13 MFCC coefficients
    mfcc_features[emotion] = mfcc

In [ ]:
# Plot MFCC heatmaps
plt.figure(figsize=(15, 5))  # Set figure size for better visuals

for i, (emotion, mfcc) in enumerate(mfcc_features.items()):
    plt.subplot(1, len(mfcc_features), i + 1)  # Create subplots dynamically
    librosa.display.specshow(mfcc, x_axis='time', sr=16000, cmap='viridis')  # Heatmap
    plt.colorbar(label='MFCC Coeff Magnitude')
    plt.title(f'MFCC Heatmap - {emotion.capitalize()}')
    plt.xlabel('Time (s)')
    plt.ylabel('MFCC Coefficients')

plt.tight_layout()
plt.show()


In [ ]:
# Example: Annotate the "happy" emotion's heatmap
emotion = "sad"
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc_features[emotion], x_axis='time', sr=16000, cmap='viridis')
plt.colorbar(label='MFCC Coeff Magnitude')
plt.title(f'MFCC Heatmap - {emotion.capitalize()}')
plt.xlabel('Time (s)')
plt.ylabel('MFCC Coefficients')

# Add a rectangle around a region of interest
plt.gca().add_patch(plt.Rectangle((0.5, 0), 0.5, 3, fill=False, edgecolor='red', linewidth=2, linestyle='--'))
plt.text(0.6, 1, 'High energy region', color='red', fontsize=10)
plt.tight_layout()
plt.show()


# PCA

In [19]:
# Define a dictionary to map emotion names to numeric labels
emotion_to_label = {emotion: idx for idx, emotion in enumerate(data_path["Emotions"].unique())}
label_to_emotion = {idx: emotion for emotion, idx in emotion_to_label.items()}

# Containers for features and labels
mfcc_features = []
emotion_labels = []

# Extract MFCC features for each sample
for _, row in data_path.iterrows():
    emotion = row['Emotions']
    file_path = row['Path']
    
    # Load audio file
    y, sr = librosa.load(file_path, sr=16000)
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)  # Aggregate across time frames
    
    # Append features and labels
    mfcc_features.append(mfcc_mean)
    emotion_labels.append(emotion_to_label[emotion])

# Convert to numpy arrays
mfcc_features = np.array(mfcc_features)
emotion_labels = np.array(emotion_labels)

In [20]:
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(mfcc_features)


In [ ]:
import matplotlib.pyplot as plt

def plot_emotion_clusters_pca(reduced_features, emotion_labels, label_to_emotion, title="Emotion Clusters (PCA)"):
    plt.figure(figsize=(8, 6))
    for label in np.unique(emotion_labels):
        indices = emotion_labels == label
        plt.scatter(
            reduced_features[indices, 0],  # First principal component
            reduced_features[indices, 1],  # Second principal component
            label=label_to_emotion[label],  # Emotion name
            alpha=0.7
        )
    plt.legend()
    plt.title(title)
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.show()

# Call the function
plot_emotion_clusters_pca(reduced_features, emotion_labels, label_to_emotion)


# Mean MFCC

In [ ]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
data = data_path

# Define a container to store MFCCs grouped by emotion
emotion_mfcc = {emotion: [] for emotion in data["Emotions"].unique()}

# Extract MFCC features for each file
for _, row in data.iterrows():
    emotion = row["Emotions"]
    file_path = row["Path"]
    
    # Load audio file
    try:
        y, sr = librosa.load(file_path, sr=16000)
        
        # Extract MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)  # Aggregate across time frames
        
        # Store the mean MFCCs for this file
        emotion_mfcc[emotion].append(mfcc_mean)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Compute mean MFCCs for each emotion
mean_mfcc_profiles = {emotion: np.mean(features, axis=0) for emotion, features in emotion_mfcc.items()}

# Create a DataFrame for visualization
mean_mfcc_df = pd.DataFrame(mean_mfcc_profiles)

# Plot mean MFCC profiles
plt.figure(figsize=(10, 6))
for emotion in mean_mfcc_df.columns:
    plt.plot(mean_mfcc_df.index, mean_mfcc_df[emotion], label=emotion)
    
plt.title("Mean MFCC Profiles Across Emotions")
plt.xlabel("MFCC Coefficients")
plt.ylabel("Mean Value")
plt.legend(title="Emotions")
plt.grid()
plt.show()


In [ ]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
data = data_path

# Container for storing time-aligned MFCCs grouped by emotion
emotion_time_mfcc = {emotion: [] for emotion in data["Emotions"].unique()}

# Define a fixed number of time frames for alignment
fixed_frames = 100

# Extract MFCC features and align them
for _, row in data.iterrows():
    emotion = row["Emotions"]
    file_path = row["Path"]
    
    # Load audio file
    try:
        y, sr = librosa.load(file_path, sr=16000)
        
        # Extract MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        
        # Align MFCCs to the fixed number of frames
        mfcc_resized = librosa.util.fix_length(mfcc, size=fixed_frames, axis=1)
        
        # Store the time-aligned MFCCs
        emotion_time_mfcc[emotion].append(mfcc_resized)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Compute mean MFCCs over time for each emotion
mean_time_mfcc = {
    emotion: np.mean(np.array(features), axis=0) for emotion, features in emotion_time_mfcc.items()
}

# Plot MFCC dynamics over time for each emotion
plt.figure(figsize=(12, 8))
for emotion, mfcc_mean in mean_time_mfcc.items():
    plt.plot(
        np.arange(fixed_frames), 
        np.mean(mfcc_mean, axis=0),  # Mean across coefficients
        label=emotion
    )
    
plt.title("MFCC Dynamics Over Time for Emotions")
plt.xlabel("Time Frames")
plt.ylabel("Mean MFCC Value")
plt.legend(title="Emotions")
plt.grid()
plt.show()


In [24]:
from scipy.stats import f_oneway

data = data_path
# Initialize a dictionary to store MFCC coefficients for each emotion
emotion_mfcc_stats = {emotion: [] for emotion in data["Emotions"].unique()}

# Loop through the dataset to extract MFCC features for each file
for _, row in data.iterrows():
    emotion = row["Emotions"]
    file_path = row["Path"]
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=16000)
        
        # Extract MFCC features
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        
        # Compute the mean MFCC coefficients (reduce time dimension)
        mfcc_mean = mfcc.mean(axis=1)
        
        # Append the MFCC mean to the respective emotion's list
        emotion_mfcc_stats[emotion].append(mfcc_mean)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
# Prepare ANOVA test results
anova_results = []
coeff_names = [f"MFCC {i+1}" for i in range(13)]
emotion_labels = list(emotion_mfcc_stats.keys())

# Loop through each MFCC coefficient to test for differences between emotions
for coeff_idx in range(13):
    # Collect data for the current MFCC coefficient across all emotions
    samples = [np.array([sample[coeff_idx] for sample in emotion_mfcc_stats[emotion]])
               for emotion in emotion_labels]
    
    # Perform ANOVA test
    f_stat, p_value = f_oneway(*samples)
    anova_results.append({"Coefficient": coeff_names[coeff_idx], "F-Stat": f_stat, "P-Value": p_value})

# Convert ANOVA results into a DataFrame for easier visualization
anova_df = pd.DataFrame(anova_results)
anova_df["Significant"] = anova_df["P-Value"] < 0.05
print(anova_df)


In [ ]:
# Compute the mean MFCC coefficients for each emotion
emotion_mfcc_means = {
    emotion: np.mean(np.array(mfccs), axis=0) for emotion, mfccs in emotion_mfcc_stats.items()
}

# Convert to a DataFrame for plotting
mean_mfcc_df = pd.DataFrame(emotion_mfcc_means, index=coeff_names)

# Plot the mean MFCC coefficients
plt.figure(figsize=(12, 6))
mean_mfcc_df.plot(kind="bar", figsize=(14, 8))
plt.title("Mean MFCC Coefficients Across Emotions")
plt.xlabel("MFCC Coefficients")
plt.ylabel("Mean Value")
plt.legend(title="Emotion", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
# Highlight significant MFCC coefficients in the plot
significant_coeffs = anova_df[anova_df["Significant"]]["Coefficient"].tolist()
print(f"Significant Coefficients: {significant_coeffs}")

plt.figure(figsize=(12, 6))
sns.barplot(data=mean_mfcc_df.T, ci="sd", palette="muted")
plt.title("Mean MFCC Coefficients with Statistical Significance")
plt.xlabel("Emotion")
plt.ylabel("MFCC Mean Value")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", title="MFCC Coefficients")
plt.tight_layout()
plt.show()


In [ ]:
data_path.Path[0]
data_path.Emotions[0]

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_path = data_path.Path[0]  # Replace with your file path
y, sr = librosa.load(audio_path, sr=16000)  # `sr=None` keeps the original sampling rate

# Plot the waveform
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y, sr=sr, alpha=0.5)
plt.title("Waveform")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

# Generate and plot the spectrogram
D = librosa.stft(y)  # Short-time Fourier Transform
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(S_db, sr=sr, x_axis="time", y_axis="hz", cmap="magma")
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.tight_layout()
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_path = "AudioData\sch.wav"  # Replace with your file path
y, sr = librosa.load(audio_path, sr=16000)  # `sr` is the sampling rate

# Specify the duration (in seconds) and start time
start_time = 5  # Start at 10 seconds (adjust as needed)
duration = 5  # Duration of the segment in seconds

# Extract the desired segment
start_sample = int(start_time * sr)
end_sample = int((start_time + duration) * sr)
y_segment = y[start_sample:end_sample]

# Plot the waveform of the segment
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y_segment, sr=sr, alpha=0.5)
plt.title(f"Waveform (Segment {start_time}-{start_time + duration}s)")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

# Generate and plot the spectrogram of the segment
D = librosa.stft(y_segment)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(S_db, sr=sr, x_axis="time", y_axis="hz", cmap="magma")
plt.colorbar(format="%+2.0f dB")
plt.title(f"Spectrogram (Segment {start_time}-{start_time + duration}s)")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.tight_layout()
plt.show()
